In [ ]:
# from langchain.llms import OpenAI -> 더 이상 지원하지 않는 방법
from langchain.chat_models import ChatOpenAI

# llm = OpenAI(model_name = "gpt-3.5-turbo-1106")
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

# a, b
b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'